In [1]:
import torch.cuda
import pickle
import numpy as np
import pandas as pd
from GRUD import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
speed_matrix = pd.read_pickle(r"C:\Users\Kleve\PycharmProjects\GRUD\GRU-D\speed_matrix_2015")[:1000]
speed_matrix

ID,d005es15036,d005es15125,d005es15214,d005es15280,d005es15315,d005es15348,d005es15410,d005es15465,d005es15531,d005es15569,...,i520es00526,i520es00560,i520es00624,i520es00684,i520es00714,i520es00746,i520es00770,i520es00861,i520es00935,i520es00972
stamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,61.939138,64.280883,62.077397,60.786423,63.120675,64.448315,63.411123,64.739481,63.009918,65.264902,...,64.092842,60.397897,62.045617,62.099860,63.555292,63.625611,62.118397,68.112571,66.567829,62.032062
2015-01-01 00:05:00,59.232527,65.082450,64.808345,65.853953,59.206229,62.496716,65.992183,64.718051,61.244073,65.608728,...,64.244069,64.091079,65.082815,59.930435,63.817700,47.836660,54.307249,59.022999,58.949034,61.212069
2015-01-01 00:10:00,61.991801,65.309123,64.803916,64.266082,62.239202,63.816610,60.196829,65.447790,63.797764,66.017157,...,59.839932,63.624790,57.179902,62.603473,64.117791,58.099941,58.923199,58.710086,56.671427,57.488732
2015-01-01 00:15:00,62.480655,65.191651,67.206597,63.988427,65.808507,64.757556,62.011448,66.334476,61.702734,65.735430,...,65.230148,66.042141,61.952397,58.193563,55.949144,60.140768,57.117960,64.368119,57.892398,64.087189
2015-01-01 00:20:00,62.490484,65.287669,67.323285,64.707409,65.708663,65.358370,65.091449,63.095048,62.186795,65.097373,...,66.005431,61.455915,62.117347,63.089581,62.961678,62.849955,54.681552,62.795588,62.545365,64.567285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-04 10:55:00,62.953463,65.479792,66.789542,62.926580,66.214513,61.453995,59.898883,62.743559,63.570794,65.315037,...,63.597624,64.829116,65.003415,60.145160,62.269784,63.727304,62.137195,67.912008,63.959334,65.231463
2015-01-04 11:00:00,60.805742,61.777731,64.164094,63.677470,63.233498,60.692208,60.873466,68.179588,66.221217,67.838200,...,65.202548,64.406568,62.804343,59.189099,59.666118,64.044282,61.896432,68.433463,69.024759,67.302542
2015-01-04 11:05:00,62.172178,65.339354,66.847034,64.752331,65.156488,62.769459,61.105867,66.001344,65.495147,66.167630,...,73.013616,64.495981,62.115572,58.646694,62.557743,63.133873,63.520706,68.425716,66.047084,66.458846


In [3]:
speed_matrix.shape

(1000, 323)

In [4]:
speed_matrix.isna().sum()

ID
d005es15036    0
d005es15125    0
d005es15214    0
d005es15280    0
d005es15315    0
              ..
i520es00746    0
i520es00770    0
i520es00861    0
i520es00935    0
i520es00972    0
Length: 323, dtype: int64

In [5]:
# step for step the PrepareDataset-Function
BATCH_SIZE = 40
seq_len = 10
pred_len = 1
train_propotion = 0.7
valid_propotion = 0.2
masking = False
mask_ones_proportion = 0.8

""" Prepare training and testing datasets and dataloaders.

Convert speed/volume/occupancy matrix to training and testing dataset.
The vertical axis of speed_matrix is the time axis and the horizontal axis
is the spatial axis.

Args:
    speed_matrix: a Matrix containing spatial-temporal speed data for a network
    seq_len: length of input sequence
    pred_len: length of predicted sequence
Returns:
    Training dataloader
    Testing dataloader
"""

' Prepare training and testing datasets and dataloaders.\n\nConvert speed/volume/occupancy matrix to training and testing dataset.\nThe vertical axis of speed_matrix is the time axis and the horizontal axis\nis the spatial axis.\n\nArgs:\n    speed_matrix: a Matrix containing spatial-temporal speed data for a network\n    seq_len: length of input sequence\n    pred_len: length of predicted sequence\nReturns:\n    Training dataloader\n    Testing dataloader\n'

In [6]:
time_len = speed_matrix.shape[0]
time_len

1000

In [7]:
speed_matrix = speed_matrix.clip(0, 100)
print(speed_matrix.shape)
speed_matrix

(1000, 323)


ID,d005es15036,d005es15125,d005es15214,d005es15280,d005es15315,d005es15348,d005es15410,d005es15465,d005es15531,d005es15569,...,i520es00526,i520es00560,i520es00624,i520es00684,i520es00714,i520es00746,i520es00770,i520es00861,i520es00935,i520es00972
stamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,61.939138,64.280883,62.077397,60.786423,63.120675,64.448315,63.411123,64.739481,63.009918,65.264902,...,64.092842,60.397897,62.045617,62.099860,63.555292,63.625611,62.118397,68.112571,66.567829,62.032062
2015-01-01 00:05:00,59.232527,65.082450,64.808345,65.853953,59.206229,62.496716,65.992183,64.718051,61.244073,65.608728,...,64.244069,64.091079,65.082815,59.930435,63.817700,47.836660,54.307249,59.022999,58.949034,61.212069
2015-01-01 00:10:00,61.991801,65.309123,64.803916,64.266082,62.239202,63.816610,60.196829,65.447790,63.797764,66.017157,...,59.839932,63.624790,57.179902,62.603473,64.117791,58.099941,58.923199,58.710086,56.671427,57.488732
2015-01-01 00:15:00,62.480655,65.191651,67.206597,63.988427,65.808507,64.757556,62.011448,66.334476,61.702734,65.735430,...,65.230148,66.042141,61.952397,58.193563,55.949144,60.140768,57.117960,64.368119,57.892398,64.087189
2015-01-01 00:20:00,62.490484,65.287669,67.323285,64.707409,65.708663,65.358370,65.091449,63.095048,62.186795,65.097373,...,66.005431,61.455915,62.117347,63.089581,62.961678,62.849955,54.681552,62.795588,62.545365,64.567285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-04 10:55:00,62.953463,65.479792,66.789542,62.926580,66.214513,61.453995,59.898883,62.743559,63.570794,65.315037,...,63.597624,64.829116,65.003415,60.145160,62.269784,63.727304,62.137195,67.912008,63.959334,65.231463
2015-01-04 11:00:00,60.805742,61.777731,64.164094,63.677470,63.233498,60.692208,60.873466,68.179588,66.221217,67.838200,...,65.202548,64.406568,62.804343,59.189099,59.666118,64.044282,61.896432,68.433463,69.024759,67.302542
2015-01-04 11:05:00,62.172178,65.339354,66.847034,64.752331,65.156488,62.769459,61.105867,66.001344,65.495147,66.167630,...,73.013616,64.495981,62.115572,58.646694,62.557743,63.133873,63.520706,68.425716,66.047084,66.458846


In [8]:
max_speed = speed_matrix.max().max()
print(f"{max_speed = }")
speed_matrix = speed_matrix / max_speed

max_speed = 78.82251882251884


In [9]:
speed_sequences, speed_labels = [], []
# takes 10 rows and tries to predict the next row
for i in range(time_len - seq_len - pred_len):
    speed_sequences.append(speed_matrix.iloc[i:i + seq_len].values)
    speed_labels.append(speed_matrix.iloc[i + seq_len:i + seq_len + pred_len].values)

speed_sequences, speed_labels = np.asarray(speed_sequences), np.asarray(speed_labels)
print(f"{speed_sequences.shape = }")
print(f"{speed_labels.shape = }")

speed_sequences

speed_sequences.shape = (989, 10, 323)
speed_labels.shape = (989, 1, 323)


array([[[0.78580511, 0.8155142 , 0.78755916, ..., 0.86412578,
         0.84452806, 0.78698401],
        [0.75146706, 0.82568346, 0.82220596, ..., 0.74880884,
         0.74787047, 0.77658098],
        [0.78647323, 0.8285592 , 0.82214977, ..., 0.744839  ,
         0.71897508, 0.72934402],
        ...,
        [0.80579476, 0.84701668, 0.84490242, ..., 0.80351446,
         0.83098214, 0.8220496 ],
        [0.79593576, 0.84487999, 0.84448317, ..., 0.85112158,
         0.82573709, 0.80689896],
        [0.79080532, 0.83470161, 0.82827742, ..., 0.82650588,
         0.78297481, 0.78055805]],

       [[0.75146706, 0.82568346, 0.82220596, ..., 0.74880884,
         0.74787047, 0.77658098],
        [0.78647323, 0.8285592 , 0.82214977, ..., 0.744839  ,
         0.71897508, 0.72934402],
        [0.79267519, 0.82706887, 0.85263194, ..., 0.81662094,
         0.73446521, 0.81305685],
        ...,
        [0.79593576, 0.84487999, 0.84448317, ..., 0.85112158,
         0.82573709, 0.80689896],
        [0.7

In [12]:
# using zero-one mask to randomly set elements to zeros
# if masking:
from collections import Counter
print('Split Speed finished. Start to generate Mask, Delta, Last_observed_X ...')
np.random.seed(1024)
Mask = np.random.choice([0, 1], size=(speed_sequences.shape),
                        p=[1 - mask_ones_proportion, mask_ones_proportion])
print(f"{Mask.shape = }")
## print(Counter(list(Mask)))
Mask

Split Speed finished. Start to generate Mask, Delta, Last_observed_X ...
Mask.shape = (989, 10, 323)


array([[[1, 1, 1, ..., 0, 1, 1],
        [1, 0, 1, ..., 1, 0, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 0, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 0, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 1, 0, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [0, 0, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 0, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 0, 1],
        [1, 1, 1, ..., 1, 1, 0]],

       ...,

       [[0, 0, 0, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 0, 0, 1],
        [1, 0, 1, ..., 1, 1, 1]],

       [[0, 1, 0, ..., 0, 1, 1],
        [1, 1, 0, ..., 1, 0, 1],
        [0, 1, 0, ..., 1, 0, 1],
        ...,
        [1, 0, 1, ..., 

In [19]:
speed_sequences = np.multiply(speed_sequences, Mask) # element wise multiplication

In [21]:
# temporal information
interval = 5  # 5 minutes
S = np.zeros_like(speed_sequences)  # time stamps
for i in range(S.shape[1]):
    S[:, i, :] = interval * i
print(f"{S.shape = }")
S

S.shape = (989, 10, 323)


array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [10., 10., 10., ..., 10., 10., 10.],
        ...,
        [35., 35., 35., ..., 35., 35., 35.],
        [40., 40., 40., ..., 40., 40., 40.],
        [45., 45., 45., ..., 45., 45., 45.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [10., 10., 10., ..., 10., 10., 10.],
        ...,
        [35., 35., 35., ..., 35., 35., 35.],
        [40., 40., 40., ..., 40., 40., 40.],
        [45., 45., 45., ..., 45., 45., 45.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [10., 10., 10., ..., 10., 10., 10.],
        ...,
        [35., 35., 35., ..., 35., 35., 35.],
        [40., 40., 40., ..., 40., 40., 40.],
        [45., 45., 45., ..., 45., 45., 45.]],

       ...,

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [10., 10., 10., ..., 10., 10., 10.

In [22]:
Delta = np.zeros_like(speed_sequences)  # time intervals
for i in range(1, S.shape[1]):
    Delta[:, i, :] = S[:, i, :] - S[:, i - 1, :]
print(f"{Delta.shape = }")
Delta

Delta.shape = (989, 10, 323)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.],
        ...,
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.],
        ...,
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.],
        ...,
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.],
        ...,
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5., 5.],
        [5., 5., 5., ..., 5., 5.

In [23]:
missing_index = np.where(Mask == 0)
print(f"{missing_index[0].shape = }") # rows
print(f"{missing_index[1].shape = }") # seq_len
print(f"{missing_index[2].shape = }") # features
missing_index

missing_index[0].shape = (638700,)
missing_index[1].shape = (638700,)
missing_index[2].shape = (638700,)


(array([  0,   0,   0, ..., 988, 988, 988], dtype=int64),
 array([0, 0, 0, ..., 9, 9, 9], dtype=int64),
 array([  6,   7,   8, ..., 290, 303, 310], dtype=int64))

In [24]:
X_last_obsv = np.copy(speed_sequences)
for idx in range(missing_index[0].shape[0]):
    # i, j, k coordinates for missing_index
    i = missing_index[0][idx]
    j = missing_index[1][idx]
    k = missing_index[2][idx]
    if j != 0 and j != 9:
        Delta[i, j + 1, k] = Delta[i, j + 1, k] + Delta[i, j, k]
    if j != 0:
        X_last_obsv[i, j, k] = X_last_obsv[i, j - 1, k]  # last observation

print(f"{X_last_obsv.shape = }")
X_last_obsv

X_last_obsv.shape = (989, 10, 323)


array([[[0.78580511, 0.8155142 , 0.78755916, ..., 0.86412578,
         0.84452806, 0.78698401],
        [0.75146706, 0.8155142 , 0.82220596, ..., 0.74880884,
         0.84452806, 0.77658098],
        [0.78647323, 0.8285592 , 0.82214977, ..., 0.744839  ,
         0.71897508, 0.72934402],
        ...,
        [0.80579476, 0.84701668, 0.84490242, ..., 0.83519411,
         0.83098214, 0.8220496 ],
        [0.79593576, 0.84487999, 0.84448317, ..., 0.85112158,
         0.82573709, 0.80689896],
        [0.79080532, 0.83470161, 0.82827742, ..., 0.82650588,
         0.82573709, 0.78055805]],

       [[0.75146706, 0.82568346, 0.82220596, ..., 0.74880884,
         0.74787047, 0.77658098],
        [0.75146706, 0.82568346, 0.82220596, ..., 0.744839  ,
         0.71897508, 0.72934402],
        [0.75146706, 0.82706887, 0.82220596, ..., 0.81662094,
         0.73446521, 0.81305685],
        ...,
        [0.79593576, 0.84487999, 0.84448317, ..., 0.85112158,
         0.82573709, 0.80689896],
        [0.7

In [25]:
Delta

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [ 5., 10.,  5., ...,  5., 10.,  5.],
        ...,
        [ 5., 10.,  5., ...,  5.,  5., 10.],
        [ 5.,  5.,  5., ..., 10.,  5.,  5.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [10., 10., 10., ...,  5.,  5.,  5.],
        ...,
        [ 5.,  5.,  5., ...,  5., 10.,  5.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [10., 10.,  5., ...,  5.,  5.,  5.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [ 5.,  5., 10., ...,  5.,  5.,  5.],
        ...,
        [ 5., 10.,  5., ...,  5., 10., 10.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [ 5.,  5.,  5., ...,  5., 10.,  5.]],

       ...,

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.],
        [ 5.,  5.,  5., ...,  5.,  5.,  5.

In [26]:
Delta = Delta / Delta.max()  # normalize

In [25]:
# shuffle and split the dataset to training and testing datasets
print('Generate Mask, Delta, Last_observed_X finished. Start to shuffle and split dataset ...')
sample_size = speed_sequences.shape[0]
index = np.arange(sample_size, dtype=int)
np.random.seed(1024)
np.random.shuffle(index)

Generate Mask, Delta, Last_observed_X finished. Start to shuffle and split dataset ...


In [27]:
speed_sequences = speed_sequences[index]
speed_labels = speed_labels[index]
print(f"{speed_sequences.shape = }")
print(f"{speed_labels.shape = }")
print(f"{speed_sequences[index].shape = }")
print(f"{speed_labels[index].shape = }")
speed_sequences

speed_sequences.shape = (989, 10, 323)
speed_labels.shape = (989, 1, 323)
speed_sequences[index].shape = (989, 10, 323)
speed_labels[index].shape = (989, 1, 323)


array([[[0.70611588, 0.        , 0.65050004, ..., 0.84362454,
         0.        , 0.8167963 ],
        [0.72705777, 0.72579915, 0.68658727, ..., 0.80532428,
         0.75358661, 0.7667995 ],
        [0.67847787, 0.69363672, 0.68083707, ..., 0.8255688 ,
         0.74567699, 0.76937745],
        ...,
        [0.7055484 , 0.71737203, 0.69975728, ..., 0.83746475,
         0.80941801, 0.81765567],
        [0.        , 0.        , 0.74939465, ..., 0.8776972 ,
         0.78532256, 0.81380398],
        [0.73974174, 0.78053697, 0.        , ..., 0.85152642,
         0.80194254, 0.80791339]],

       [[0.79028006, 0.86027537, 0.86417356, ..., 0.82586085,
         0.76697388, 0.        ],
        [0.79755329, 0.79207487, 0.83163156, ..., 0.86052928,
         0.85750149, 0.82749548],
        [0.81227519, 0.85619114, 0.81318239, ..., 0.        ,
         0.82224776, 0.80791289],
        ...,
        [0.        , 0.8071228 , 0.84952489, ..., 0.86054073,
         0.7925461 , 0.78772114],
        [0.7

In [28]:
# if masking:
X_last_obsv = X_last_obsv[index]
print(f"{X_last_obsv.shape = }")
X_last_obsv

X_last_obsv.shape = (989, 10, 323)


array([[[0.        , 0.85130737, 0.        , ..., 0.86987671,
         0.85308397, 0.85710211],
        [0.81600619, 0.84238178, 0.84162405, ..., 0.87107149,
         0.8231116 , 0.84689024],
        [0.81555852, 0.83593706, 0.81476465, ..., 0.87107149,
         0.82711516, 0.83379631],
        ...,
        [0.8187461 , 0.84530259, 0.81960787, ..., 0.86196815,
         0.85373112, 0.84207251],
        [0.80581439, 0.82644784, 0.77287547, ..., 0.87056554,
         0.84002186, 0.84207251],
        [0.81888311, 0.85024647, 0.82119697, ..., 0.87056554,
         0.83020245, 0.84959137]],

       [[0.        , 0.83517903, 0.83796527, ..., 0.85712961,
         0.83459555, 0.80998336],
        [0.77043481, 0.83517903, 0.82821808, ..., 0.8431212 ,
         0.83459555, 0.80998336],
        [0.78032065, 0.82164276, 0.82004044, ..., 0.85921255,
         0.81753829, 0.82600783],
        ...,
        [0.79083477, 0.84022039, 0.85614528, ..., 0.85226209,
         0.80148175, 0.78629474],
        [0.7

In [29]:
Mask = Mask[index]
print(f"{Mask.shape = }")
Mask

Mask.shape = (989, 10, 323)


array([[[0, 1, 0, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 1, 1],
        ...,
        [1, 1, 0, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 0, 1, 1]],

       [[0, 1, 1, ..., 1, 1, 1],
        [1, 0, 1, ..., 1, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 0, ..., 0, 1, 0],
        [1, 1, 1, ..., 0, 1, 1]],

       [[0, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 0, 1],
        ...,
        [1, 1, 1, ..., 1, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       ...,

       [[1, 1, 1, ..., 1, 1, 1],
        [0, 1, 1, ..., 1, 1, 1],
        [0, 1, 1, ..., 0, 1, 1],
        ...,
        [1, 1, 1, ..., 0, 0, 1],
        [1, 1, 0, ..., 1, 1, 1],
        [1, 0, 1, ..., 0, 1, 0]],

       [[1, 1, 0, ..., 0, 1, 1],
        [1, 1, 0, ..., 1, 0, 1],
        [1, 1, 1, ..., 1, 1, 0],
        ...,
        [1, 1, 0, ..., 

In [31]:
Delta = Delta[index]
print(f"{Delta.shape = }")
Delta

Delta.shape = (989, 10, 323)


array([[[0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125],
        [0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125],
        ...,
        [0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125],
        [0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125],
        [0.25 , 0.25 , 0.125, ..., 0.125, 0.125, 0.125]],

       [[0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125],
        [0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125],
        ...,
        [0.125, 0.375, 0.125, ..., 0.125, 0.125, 0.25 ],
        [0.25 , 0.125, 0.125, ..., 0.125, 0.125, 0.125],
        [0.125, 0.125, 0.125, ..., 0.125, 0.25 , 0.25 ]],

       [[0.   , 0.   , 0.   , ..., 0.   , 0.   , 0.   ],
        [0.125, 0.125, 0.125, ..., 0.125, 0.125, 0.125],
        [0.25 , 0.25 , 0.125, ..., 0.125, 0.125, 0.25 ],
        ...,
        [0.125, 0.125, 0.125, ..., 0.125, 0.25 , 0.375],
        [0.125, 0.125, 0.25 , ..., 0.25 , 0.1

In [32]:
speed_sequences = np.expand_dims(speed_sequences, axis=1)

X_last_obsv = np.expand_dims(X_last_obsv, axis=1)
Mask = np.expand_dims(Mask, axis=1)
Delta = np.expand_dims(Delta, axis=1)

print(f"{X_last_obsv.shape = }")
print(f"{Mask.shape = }")
print(f"{Delta.shape = }")

X_last_obsv.shape = (989, 1, 10, 323)
Mask.shape = (989, 1, 10, 323)
Delta.shape = (989, 1, 10, 323)


In [34]:
dataset_agger = np.concatenate((speed_sequences, X_last_obsv, Mask, Delta), axis=1)
# => 4 dimensions because of four dimensional space as raw data, Mask, Delta and last oberserved data
print(f"{dataset_agger.shape = }")
dataset_agger

dataset_agger.shape = (989, 4, 10, 323)


array([[[[0.70611588, 0.        , 0.65050004, ..., 0.84362454,
          0.        , 0.8167963 ],
         [0.72705777, 0.72579915, 0.68658727, ..., 0.80532428,
          0.75358661, 0.7667995 ],
         [0.67847787, 0.69363672, 0.68083707, ..., 0.8255688 ,
          0.74567699, 0.76937745],
         ...,
         [0.7055484 , 0.71737203, 0.69975728, ..., 0.83746475,
          0.80941801, 0.81765567],
         [0.        , 0.        , 0.74939465, ..., 0.8776972 ,
          0.78532256, 0.81380398],
         [0.73974174, 0.78053697, 0.        , ..., 0.85152642,
          0.80194254, 0.80791339]],

        [[0.        , 0.85130737, 0.        , ..., 0.86987671,
          0.85308397, 0.85710211],
         [0.81600619, 0.84238178, 0.84162405, ..., 0.87107149,
          0.8231116 , 0.84689024],
         [0.81555852, 0.83593706, 0.81476465, ..., 0.87107149,
          0.82711516, 0.83379631],
         ...,
         [0.8187461 , 0.84530259, 0.81960787, ..., 0.86196815,
          0.85373112, 0.8

In [35]:
train_index = int(np.floor(sample_size * train_propotion))

valid_index = int(np.floor(sample_size * (train_propotion + valid_propotion)))

print(f"{train_index = }")
print(f"{valid_index = }")

train_index = 692
valid_index = 890


In [36]:
# if we do not mask he does not know what dataset_agger is
if masking:
    train_data, train_label = dataset_agger[:train_index], speed_labels[:train_index]
    valid_data, valid_label = dataset_agger[train_index:valid_index], speed_labels[train_index:valid_index]
    test_data, test_label = dataset_agger[valid_index:], speed_labels[valid_index:]
else:
    train_data, train_label = speed_sequences[:train_index], speed_labels[:train_index]
    valid_data, valid_label = speed_sequences[train_index:valid_index], speed_labels[train_index:valid_index]
    test_data, test_label = speed_sequences[valid_index:], speed_labels[valid_index:]

In [37]:
# Creating DataLoader

train_data, train_label = torch.Tensor(train_data), torch.Tensor(train_label)
valid_data, valid_label = torch.Tensor(valid_data), torch.Tensor(valid_label)
test_data, test_label = torch.Tensor(test_data), torch.Tensor(test_label)

train_dataset = utils.TensorDataset(train_data, train_label)
valid_dataset = utils.TensorDataset(valid_data, valid_label)
test_dataset = utils.TensorDataset(test_data, test_label)

train_dataloader = utils.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
valid_dataloader = utils.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_dataloader = utils.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)


In [39]:
X_mean = np.mean(speed_sequences, axis=0)
print(f"{X_mean.shape = }")
X_mean

X_mean.shape = (1, 10, 323)


array([[[0.62128245, 0.64754571, 0.6480787 , ..., 0.6621371 ,
         0.62436915, 0.65621634],
        [0.61469396, 0.66923337, 0.65653006, ..., 0.67751844,
         0.65071576, 0.63862633],
        [0.63484673, 0.66580185, 0.66496442, ..., 0.66843054,
         0.64023624, 0.64903192],
        ...,
        [0.62246886, 0.67130866, 0.66496006, ..., 0.64091315,
         0.63316868, 0.62486245],
        [0.61726001, 0.66122948, 0.6418147 , ..., 0.64792456,
         0.64840582, 0.65024826],
        [0.62964152, 0.65850837, 0.64915444, ..., 0.66605762,
         0.62514693, 0.63784978]]])

In [40]:
print('Finished')

Finished
